In [2]:
import os
import scanpy as sc
import anndata as an
import pandas as pd

### Распаковка tar.gz файла

In [3]:
import tarfile

# open file
file = tarfile.open('tar_data/macaque.tar.gz')
# extracting file
file.extractall('data/spatial_data/')
file.close()

### Объединим файлы с разметкой и сохраним в anndata objects

In [12]:
# Define paths and get sample_ids
obj_list = list()
file_list = os.listdir('data/spatial_data/chimp/')
file_list

['chimp_11454', 'chimp_13302', 'chimp_13309', 'chimp_j11', 'chimp_j8']

In [13]:
# Read each table and append to our list
for file in file_list:
    print(file)
    ann = pd.read_csv('data/clusters/' + file + '.csv', index_col=0)
    ann.columns = ['label']
    tmp = sc.read_visium('data/spatial_data/chimp/'+ file + '/outs')
    tmp.var_names_make_unique()
    tmp.obs['label'] = ann.label
    tmp.obs['sample_id'] = file
    obj_list.append(tmp)

chimp_11454


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_13302


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_13309


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_j11


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


chimp_j8


C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


### Объединим anndata объекты

In [14]:
adata = an.concat( obj_list , merge='same',uns_merge="unique" )
adata.var_names_make_unique()

C:\Users\aleks\anaconda3\envs\scanorama39\lib\site-packages\anndata\_core\anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [15]:
adata.uns['spatial'].keys()

dict_keys(['11454', 'chimp_13302', 'chimp_13309', 'chimp_j11', 'chimp_j8'])

In [9]:
adata.uns['spatial']['chimp_11454'] = adata.uns['spatial'].pop('11454')

In [10]:
adata.uns['spatial'].keys()

dict_keys(['macaque_0701', 'macaque_0703', 'macaque_0704', 'macaque_nb0904'])

### Сохраним файлы

In [11]:
os.makedirs('results', exist_ok=True)
os.makedirs('results/spatial_transcriptomics', exist_ok=True)
adata.write_h5ad('results/spatial_transcriptomics/macaque.h5ad')